In [1]:
import nltk as nl
import pandas as pd
import re
import random
import string
import joblib
from nltk.stem import WordNetLemmatizer 
from nltk import FreqDist
from math import floor
from nltk.stem import PorterStemmer
from nltk import classify
from nltk import NaiveBayesClassifier

In [2]:
try: 
    positive_messages= []
    negative_messages = []
    
    dataset = pd.read_csv("dataset.csv")
    
#     print(dataset.shape)
#     print(dataset.isna().sum())
    
#     dataset = dataset.dropna()

#     print(dataset.shape)
    
    messages = dataset['messages']
    label = dataset['label']
#     dataset.to_csv("editied_sentiment_tweets3.csv")
    length = dataset.shape
    
#     print(dataset.isna().sum())
#     print("length:",length)
    for i in range(length[0]):
        if label[i] == 1:
            positive_messages.append(messages[i])
        elif label[i] == -1:
            negative_messages.append(messages[i])

# for i in range(len(label)):
    

except Exception as e:
    print(e)

In [3]:
def remove_url(input_data):
    
    url_pattern = r'http\S+'
    if re.search(url_pattern,input_data):
        input_data = re.sub(url_pattern,'',input_data)
            
    return input_data

def remove_username(input_data):
    
    username_pattern = '@[^\s]+'
    if re.search(username_pattern,input_data):
        input_data = re.sub(username_pattern,'',input_data)
            
    return input_data

def tokenize(input_data):
    return nl.word_tokenize(input_data)

def remove_character(input_data):
    if input_data not in string.punctuation:
        return input_data

def remove_stopwords(input_data):
    stop_words = set(nl.corpus.stopwords.words("English"))
    if input_data not in stop_words:
        return input_data

def stemming(input_data):
    ps = PorterStemmer()
    store = ps.stem(input_data)
    return store

In [9]:
def remove_noise(input_file,mode):
    
    filtered_list = []
    tokenized_list = []
    stemmed_list = []
    
    if mode == 'list':
        for sentences in input_file:
            tokenized_list.append(tokenize(remove_username(remove_url(sentences))))
        for sentences in tokenized_list:
            for words in sentences:
                filtered_list.append(remove_stopwords(remove_character(words)))
                
        filtered_list = list(filter(None,filtered_list))
        for words in filtered_list:
            stemmed_list.append(stemming(words))
    else:
        tokenized = tokenize(remove_username(remove_url(input_file)))     
        for sentences in tokenized:
#             for words in sentences:
            filtered_list.append(remove_stopwords(remove_character(sentences)))
            filtered_list = list(filter(None,filtered_list))
        for words in filtered_list:
            stemmed_list.append(stemming(words))
            
    return stemmed_list

In [10]:
def create_dictionary(input_list,label,mode):
    list_ = []
    dictionary = {}
    
    if mode == 'train':
        for twts in input_list:
            store = [{twts:True},label]
            list_.append(store)
        return list_
    else:
        for twts in input_list:
            dictionary[twts] = True
        return dictionary

In [11]:
def dataset_distribution(total_data,train_total):

    train = floor(total_data*train_total)
    test = total_data-train
    
    return train

In [12]:
%%time
noise_free_positive = remove_noise(positive_messages,'list')
noise_free_negative = remove_noise(negative_messages,'list')

Wall time: 46min 27s


In [13]:
%%time
positive_dictionary = create_dictionary(noise_free_positive,'positive','train')
negative_dictionary = create_dictionary(noise_free_negative,'negative','train')

Wall time: 5.41 s


In [14]:
# joblib.dump(noise_free_positive,'pre_proessed_positive_messages')
# joblib.dump(noise_free_negative,'pre_proessed_negative_messages')

['pre_proessed_negative_messages']

In [15]:
total_dataset = positive_dictionary + negative_dictionary
random.shuffle(total_dataset)

In [16]:
train_amount = dataset_distribution(len(total_dataset),0.8)
train = total_dataset[:train_amount]
test = total_dataset[train_amount:]

In [17]:
model = NaiveBayesClassifier.train(train)
#model = joblib.load("sentimental_analysis_model.sav")

In [18]:
#new_model = model.train(train)

In [19]:
print("Accuracy is:", classify.accuracy(model, test))

Accuracy is: 0.6830403698754526


In [20]:
# joblib.dump(new_model,"sentimental_analysis_model.sav")

# model1 = joblib.load("sentimental_analysis_model.sav")

# msg = "I ordered just once from TerribleCo, they screwed up, never used the app again."
# store = remove_noise(msg,'sentence')
# store1 = create_dictionary(store,'','sentence')
# # temp_dictionary = {}
# # for words in store:
# #     temp_dictionary[words] = True


# store1

# model1.classify(store1)

In [62]:
msg = ""
msg = remove_noise(msg,'sentence')

msg = create_dictionary(msg,'','sentence')

model.classify(msg)

'positive'